In [ ]:
# read in patient key
root = UF.DataRoot(2)
LimbusKey = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\LimbusKey.csv")
AllKey = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\AllPatientKey_s.csv")

# Loop through treatments
treatments = AllKey["Treatment"].unique()
new_key = pd.DataFrame()
for t in treatments:
    AllKey_t = AllKey[AllKey["Treatment"] == t]
    LimbusKey_t = LimbusKey[LimbusKey["Treatment"] == t]

    # Loop through patients
    patIDs = LimbusKey_t["PatID"].unique()

    for pat in patIDs:
        pat_L = LimbusKey_t[LimbusKey_t["PatID"] == pat]
        pat_A = AllKey_t[AllKey_t["PatID"] == pat]
        print(pat_A)
        print(pat_L)
        # add date and days column from all to limbus key based on Scan
        for i in range(len(pat_L)):
            scan = pat_L.iloc[i]["Scan"]
            fraction = pat_A.iloc[i]["Fraction"]
            date = pat_A[pat_A["Scan"] == scan]["Date"].values[0]
            days = pat_A[pat_A["Scan"] == scan]["Days"].values[0]
            pat_L.at[pat_L.index[i], "Date"] = str(date)
            pat_L.at[pat_L.index[i], "Days"] = int(days)
            pat_L.at[pat_L.index[i], "Fraction"] = int(fraction)

        pat_L["Days"] = pat_L["Days"].astype(int)
        # sort by days
        pat_L = pat_L.sort_values(by=["Fraction"])

    # add to new key
        new_key = new_key.append(pat_L)

new_key.to_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\LimbusKey_s.csv", index=False)


        
        



In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

csvs = os.listdir("E:\\Aaron\ProstateMRL\Data\Paper1\HM-FSTP\\Longitudinal\ClusterLabels\\")
#csvs = [csv for csv in csvs if "HM" in csv]

fts_s = pd.read_csv("E:\\Aaron\ProstateMRL\Data\Paper1\\HM-FSTP\\Features\\SelectedFeatures_Longitudinal.csv")
fts_s = fts_s["Feature"].values
for csv in csvs:
    df = pd.read_csv("E:\\Aaron\ProstateMRL\Data\Paper1\\HM-FSTP\\Longitudinal\ClusterLabels\\" + csv)
    pat = str(csv)[:-4]
    df["Selected"] = df["Feature"].apply(lambda x: x in fts_s)
    # print where selected is True
    df = df[['Feature', 'Cluster', 'Fraction', 'FeatureChange', 'Selected']]
    df['Feature'] = df['Feature'].str.replace('original_', '')
    clusters = df["Cluster"].unique()
    clusters = sorted(clusters, key=lambda x: int(x))
    for c in clusters:
        df_c = df[df["Cluster"] == c]
        df_c = df_c.sort_values(by = ["Fraction"])

        # get selected features
        selected_fts = df_c[df_c["Selected"] == True]["Feature"].unique()
        if len(selected_fts) == 0:
            sf_str = "No features selected"
        
        number_fts = "Total number of feature(s) in Cluster {}: {}\nNumber of selected features: {}\n".format(c, df_c["Feature"].nunique(), len(selected_fts) )
        text_str = selected_fts
        text_str = '\n'.join(text_str)
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.7)
        fts = df_c["Feature"].values
        plt.figure(figsize=(10, 10))
        plt.title(" Cluster " + str(c))
        sns.set_theme(style="whitegrid")
        sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
        for ft in fts:
            df_ft = df_c[df_c["Feature"] == ft]
            values = df_ft["FeatureChange"].values
            fractions = df_ft["Fraction"].values
            colour = "blue" if df_ft["Selected"].values[0] else "grey"
            l = df_ft["Feature"].values[0]
            plt.plot(fractions, values, label = l, color = colour)
            #plt.scatter(fractions, values, color = colour)
        plt.xlabel("Fraction", fontsize = 20)
        plt.ylabel("Feature Change", fontsize = 20)
        plt.xticks(np.arange(1, 5.1, 1))
        plt.xlim(1, 5)
        #plt.ylim(-1, 1)
        # add text box
        #plt.text(0.05, 0.95, (number_fts + text_str), transform=plt.gca().transAxes, fontsize=14, verticalalignment='top', bbox=props)
        
        #plt.legend(title = "Feature Selected", bbox_to_anchor=(1, 0.6), labels = ["Yes", "No"])
        plt.title("Cluster " + str(c), fontsize = 30)
        plt.savefig("E:\\Aaron\ProstateMRL\Data\Paper1\HM-FSTP\Longitudinal\ClusterPlots\\Run2_" + str(pat) + "_Cluster" + str(c) + ".png", bbox_inches = "tight")
        #plt.show()
        plt.close()






In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

dir = "E:\\Aaron\\ProstateMRL\\Data\\Paper1\\NormSummary\\"

files = os.listdir(dir)

result = pd.DataFrame()

for file in files:
    # loop through txt files and get lines that contain Before:
    with open(dir + file, "r") as f:
        lines = f.readlines()
        # filter so only lines with Before: are kept
        # get line with 
        lines = [line for line in lines if "Longitudinal" in line]

        r_lines = [line for line in lines if "Removed:" in line]
        s_line = [line for line in lines if "Selected:" in line]
        
        ICC = int(r_lines[0][-3:])
        Vol = int(r_lines[1][-3:])
        Sel = int(s_line[0][-3:])

        result = result.append({"Norm": file[:-4], "ICC": ICC, "Volume": Vol, "Selected": Sel}, ignore_index=True) 
result = result.melt(id_vars=["Norm"], var_name="Stage", value_name="Number")

# order result so plot is in correct order
result["Norm"] = pd.Categorical(result["Norm"], ["Raw", "HM-FS", "HM-TP", "HM-FSTP", "Med-Pros", "Med-Psoas"])
result["Stage"] = pd.Categorical(result["Stage"], ["ICC", "Volume", "Selected"])

# bar plot results
plt.figure(figsize=(10, 10))
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.barplot(x="Norm", y="Number", data=result, hue="Stage")
plt.xlabel("Normalisation", fontsize = 20)
plt.ylabel("Number of Features", fontsize = 20)
plt.title("Number of Features Removed or Selected", fontsize = 26)



In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


df = pd.read_csv("E:\\Aaron\ProstateMRL\Code\PatKeys\\MedianSignalNorm.csv")
df = df.drop(columns=["Unnamed: 0"])

# pick a random patient
pat = df["PatID"].sample().values[0]

# change Mask variables to Prostate and Psoas
df["Mask"] = df["Mask"].apply(lambda x: "Prostate" if x == "shrunk_pros" else "Psoas")
df = df[df["PatID"] == pat]
# plot signal over time for each normalisation in facet grid
g = sns.FacetGrid(df, col="Norm", hue="Mask", col_wrap=3, height=5, aspect=1.5)
g.map(sns.lineplot, "Scan", "Median", marker="o")
g.set_axis_labels("Fraction", "Median Signal")
g.set_titles("{col_name}")
g.add_legend()
plt.suptitle("Median Signal for Patient " + str(pat), fontsize = 30)
plt.subplots_adjust(top=0.9)
plt.show()



FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Aaron\\ProstateMRL\\Code\\PatKeys\\MedianSignalNorm.csv'

In [11]:
import pandas as pd
import scipy.cluster.hierarchy as spch
import os
from tqdm import tqdm

patIDs = ['653', '713', '752', '826', '1088', '1089', '1118', '1303', '1307', '1464', '1029',
 '1302', '1431', '1481', '1540', '1553', '1601', '1642', '829', '955']


def ClusterCheck(df, fts, t_val, df_DM):
        df_c = df
        df_new = pd.DataFrame()
        # feature names
        df_new["FeatureName"] = fts
        # cluster labels
        c = df_c["Cluster"].values[0]
        
        # need to filter distance matrix to only include features in cluster
        df_DM_c = df_DM[fts]
        # only keep features in cluster
        df_DM_c = df_DM_c[df_DM_c.index.isin(fts)]
        
        # convert to numpy array
        arr_DM_c = df_DM_c.to_numpy()
        
        # cluster
        df_new["Cluster"] = spch.fclusterdata(arr_DM_c, t=t_val, criterion="distance", method="ward")
        df_new["Cluster"] = c*100 + df_new["Cluster"]
        df_new["Cluster"] = df_new["Cluster"].astype(int)
        df_new["NumFts"] = df_new.groupby("Cluster")["Cluster"].transform("count")
        number_fts = df_new["NumFts"].unique()
        fts_check = df_new.loc[df_new["NumFts"] > 10]["FeatureName"].values
        #print(t_val, number_fts)#, df_new)
        return number_fts, df_new, fts_check


def ClusterFeatures(DataRoot, Norm, s_t_val, output):
    root = DataRoot
    DM_dir = root + "\\Aaron\\ProstateMRL\\Data\\Paper1\\" + Norm + "\\Longitudinal\\DM\\csvs\\"
    out_dir = root + "\\Aaron\\ProstateMRL\\Data\\Paper1\\"+ Norm + "\\Longitudinal\\ClusterLabels2\\"

    patIDs = ['653', '713', '752', '826', '1088', '1089', '1118', '1303', '1307', '1464', '1029',
 '1302', '1431', '1481', '1540', '1553', '1601', '1642', '829', '955']

    cluster_method = "weighted"

    for pat in tqdm(patIDs):
        df_DM = pd.read_csv(DM_dir + pat + ".csv")
        df_DM.set_index("Unnamed: 0", inplace=True)
        arr_DM = df_DM.to_numpy()
        fts = df_DM.columns

        # create temp df to hold ft name and label
        df_labels = pd.DataFrame()
        df_labels["FeatureName"] = fts

        # cluster function using DM, need to experiment with t_val and method
        df_labels["Cluster"] = spch.fclusterdata(arr_DM, t=s_t_val, criterion="distance", method=cluster_method)
        df_labels.set_index("FeatureName", inplace=True)
        
        # check number of features in each cluster
        df_labels["NumFts"] = df_labels.groupby("Cluster")["Cluster"].transform("count")
        df_labels["Cluster"] = df_labels["Cluster"].astype(int)
        print(df_labels.loc[df_labels["NumFts"] > 10])
        # loop through clusters 
        for c in df_labels["Cluster"].unique():
                df_c = df_labels[df_labels["Cluster"] == c]
                number_fts = len(df_c)
                # check numnber of features in cluster
                if number_fts > 10:
                        # if more than 10 features in cluster, reduce t_val and recluster
                        t_val = s_t_val - 0.2
                        check_fts = df_c["FeatureName"].values
                        number_fts, df_labels2 = ClusterCheck(df_c, check_fts, t_val, df_DM)
                        tries = 1
                        new_fts = df_labels2["FeatureName"].unique()
                        df_labels.loc[new_fts, "Cluster"] = df_labels2["Cluster"].values
                        df_labels["NumFts"] = df_labels.groupby("Cluster")["Cluster"].transform("count")

                        while number_fts.max() > 10:
                                t_val = t_val - 0.2
                                tries += 1
                                print("Tries: {} T_val: {}".format(tries, t_val))
                                number_fts, df_labels2 = ClusterCheck(df_c, check_fts, t_val, df_DM)
                                print(df_labels2)

                        print(df_labels2)


        # read in df with ft vals and merge
        ft_vals = pd.read_csv(root +"Aaron\\ProstateMRL\\Data\\Paper1\\"+ Norm + "\\Features\\Longitudinal_fts_pVol.csv")
        ft_vals["PatID"] = ft_vals["PatID"].astype(str)
        pat_ft_vals = ft_vals[ft_vals["PatID"] == pat]
        pat_ft_vals = pat_ft_vals.merge(df_labels, left_on="Feature", right_on="FeatureName")

        # output is feature values w/ cluster labels
        pat_ft_vals.to_csv(out_dir + pat + ".csv")


ClusterFeatures("E:\\", "HM-FS", 2, False)
    
    

  0%|          | 0/20 [00:00<?, ?it/s]

                                                 Cluster  NumFts
FeatureName                                                     
original_firstorder_Entropy                            6      24
original_firstorder_InterquartileRange                 6      24
original_firstorder_MeanAbsoluteDeviation              6      24
original_firstorder_RobustMeanAbsoluteDeviation        6      24
original_firstorder_Variance                           6      24
original_glcm_ClusterTendency                          6      24
original_glcm_Contrast                                 6      24
original_glcm_Correlation                              6      24
original_glcm_DifferenceAverage                        6      24
original_glcm_DifferenceEntropy                        6      24
original_glcm_Imc1                                     6      24
original_glcm_Imc2                                     6      24
original_glcm_InverseVariance                          6      24
original_glcm_JointEntrop

KeyError: 'FeatureName'

In [ ]:
import os
import pandas as pd

dir1 = "E:\\Aaron\\ProstateMRL\\Data\\Paper1\\HM-FS\\Longitudinal\\ClusterLabels\\"
dir2 = "E:\\Aaron\\ProstateMRL\\Data\\Paper1\\HM-FS\\Longitudinal\\ClusterLabels2\\"

files = os.listdir(dir)

for file in files:
    df1 = pd.read_csv(dir1 + file)
    df2 = pd.read_csv(dir + file)
    print("Pat: {} - Clusters1: {} - Clusters2: {}".format(file[:-4], len(df1["Cluster"].unique()), len(df2["Cluster"].unique())))
    